Model

In [1]:
import random
import numpy as np
from gmab_model import  read_shock_data
import matplotlib.pyplot as plt

import gmab_model
from gmab_model import set_world
from gmab_model import vec_rollforward, vec_GMAB, vec_GMAB_ratchet
from gmab_model import delta, rho
from gmab_model import value_by_rep, value_by_error, value_by_precision
from gmab_model import shock_value, write_shock_data, read_shock_data


def averages(file, N, delrho, policy, R):
    world = read_shock_data(file)[0] + (None,)
    shock_dat = read_shock_data(file)[1]

    range_r=21                                          # number of r shocks
    range_s=31                                          # number of s shocks

    SR=np.array([[shock_dat[r+s*range_r][0] for r in range(range_r)] for s in range(range_s)])
    V=np.array([[shock_dat[r+s*range_r][2][0] for r in range(range_r)] for s in range(range_s)])
    
    a0, b0, a1, b1, a2, b2, a3, b3, a4, b4 = 2, 2, 4, 20, 16, 11, 28, 0, 28, 16
    
    t_max = 10 # t in years
    dt = 1/252
    args = 0,10,np.ones(int(t_max/dt)+1),R,0,.01
        
    shocks = [tuple(SR[a0, b0]),tuple(SR[a1, b1]),tuple(SR[a2, b2]),tuple(SR[a3, b3]),tuple(SR[a4, b4])]
    raw = shock_value(world, shocks, value_by_rep, N ,delrho, .001, policy, *args)

    B = np.zeros(5)
    for i in range(5):
        B[i] = raw[i][2][0]
        
    MCNoise = (abs(V[a0,b0]-B[0]) + abs(V[a1,b1]-B[1]) + abs(V[a2,b2]-B[2])+ abs(V[a3,b3]-B[3])+ abs(V[a4,b4]-B[4]))/5
    
    P = np.zeros((5,2), dtype = int)
    P[0] = a0, b0
    P[1] = a1, b1
    P[2] = a2, b2
    P[3] = a3, b3
    P[4] = a4, b4

    srp = np.zeros((5,2), dtype = float)
    for i in range(5):
        for j in range(2):
            srp[i,j] = SR[P[i,0],P[i,1],j]
        
    Q = (srp-srp[0])[1:]

    W = np.zeros((4,4),dtype=float)
    for i in range(4):
        W[i] = Q[i,0], Q[i,1], Q[i,0]**2, Q[i,0]*Q[i,1]

    s0 , r0 = SR[P[0,0],P[0,1]]
    v0 = B[0]

    C = B[1:]- v0
    L = np.linalg.solve(W, C)
    
    def estimate(sx,rx):
        return v0 + L[0] * (sx-s0) + L[1] * (rx-r0) + L[2] * (sx-s0)**2 + L[3] * (sx-s0) * (rx-r0)
    
    predicts = np.array([[ estimate(SR[s,r,0],SR[s,r,1]) for r in range(range_r)]for s in range(range_s)])
    
    percent = np.sum(np.abs(V - predicts)/V) / (range_s * range_r)
    
    return percent

def percent_error(file, delrho, policy, R, N, cycles):
    av_percent = 0
    for i in range(cycles):
        av_percent += averages(file, N, delrho, policy, R) * 100

    av_percent /= cycles
    return av_percent

Delta with ratchet

In [58]:
def MC_delta_ratchet(MC):
    av_percent = 0
    for sample in ['sample2_delta_.csv','sample3_delta_.csv', 'sample4_delta_.csv', 'sample5_delta_.csv']:
        av_percent += percent_error(sample,delta, vec_GMAB_ratchet, 0.045, MC, 35)
    av_percent /= 4
    return (20*MC,av_percent)

In [59]:
[MC_delta_ratchet(MC) for MC in range(100, 2000, 200)]

[(2000, 3.9291800649347577),
 (6000, 2.3157696061765236),
 (10000, 1.6898969832355544),
 (14000, 1.5380746871123379),
 (18000, 1.292823006158918),
 (22000, 1.1843727079610231),
 (26000, 1.0424807850806528),
 (30000, 0.9941303866659221),
 (34000, 0.9405395007581159),
 (38000, 0.8704619049271584)]

Rho with ratchet

In [56]:
def MC_rho_ratchet(MC):
    av_percent = 0
    for sample in ['sample2_rho_.csv','sample3_rho_.csv', 'sample4_rho_.csv', 'sample5_rho_.csv']:
        av_percent += percent_error(sample, rho, vec_GMAB_ratchet, 0.045, MC, 35)
    av_percent /= 4
    return (20*MC,av_percent)

In [57]:
[MC_rho_ratchet(MC) for MC in range(100, 2000, 200)]

[(2000, 1.7916492329875517),
 (6000, 1.0780971320865025),
 (10000, 0.8608442450462597),
 (14000, 0.714551497811651),
 (18000, 0.6517185452973335),
 (22000, 0.5995404032381145),
 (26000, 0.5257250676155882),
 (30000, 0.48312410603231776),
 (34000, 0.4579180015993267),
 (38000, 0.44490003075633683)]

Delta without ratchet

In [48]:
def MC_delta_gmab(MC):
    av_percent = 0
    for sample in ['sample6_delta_.csv','sample7_delta_.csv', 'sample8_delta_.csv', 'sample9_delta_.csv']:
        av_percent += percent_error(sample,delta, vec_GMAB, 0.038, MC, 35)
    av_percent /= 4
    return (20*MC,av_percent)

In [49]:
[MC_delta_gmab(MC) for MC in range(100, 2000, 200)]

[(100, 3.5413067037672517),
 (300, 2.092011450588231),
 (500, 1.5424414219858902),
 (700, 1.3635568540210508),
 (900, 1.142939530601727),
 (1100, 1.0745364381477485),
 (1300, 0.9643780108619049),
 (1500, 0.8805876480450663),
 (1700, 0.861526307734736),
 (1900, 0.785490232708578)]

In [55]:
b = [(100, 3.5413067037672517),
 (300, 2.092011450588231),
 (500, 1.5424414219858902),
 (700, 1.3635568540210508),
 (900, 1.142939530601727),
 (1100, 1.0745364381477485),
 (1300, 0.9643780108619049),
 (1500, 0.8805876480450663),
 (1700, 0.861526307734736),
 (1900, 0.785490232708578)]
[(20*i,j) for (i,j) in b]

[(2000, 3.5413067037672517),
 (6000, 2.092011450588231),
 (10000, 1.5424414219858902),
 (14000, 1.3635568540210508),
 (18000, 1.142939530601727),
 (22000, 1.0745364381477485),
 (26000, 0.9643780108619049),
 (30000, 0.8805876480450663),
 (34000, 0.861526307734736),
 (38000, 0.785490232708578)]

Rho without ratchet

In [50]:
def MC_rho_gmab(MC):
    av_percent = 0
    for sample in ['sample6_rho_.csv','sample7_rho_.csv', 'sample8_rho_.csv', 'sample9_rho_.csv']:
        av_percent += percent_error(sample,rho, vec_GMAB, 0.038, MC, 35)
    av_percent /= 4
    return (20*MC,av_percent)

In [51]:
[MC_rho_gmab(MC) for MC in range(100, 2000, 200)]

[(100, 3.7523949858140266),
 (300, 2.2686008231528234),
 (500, 1.614042100466718),
 (700, 1.319706634350028),
 (900, 1.3140606504024448),
 (1100, 1.1304517058734014),
 (1300, 0.9818072836881715),
 (1500, 0.8901387149131978),
 (1700, 0.8519051070288903),
 (1900, 0.7947895999664009)]

In [54]:
a=[(100, 3.7523949858140266),
 (300, 2.2686008231528234),
 (500, 1.614042100466718),
 (700, 1.319706634350028),
 (900, 1.3140606504024448),
 (1100, 1.1304517058734014),
 (1300, 0.9818072836881715),
 (1500, 0.8901387149131978),
 (1700, 0.8519051070288903),
 (1900, 0.7947895999664009)]
[(20*i,j) for (i,j) in a]

[(2000, 3.7523949858140266),
 (6000, 2.2686008231528234),
 (10000, 1.614042100466718),
 (14000, 1.319706634350028),
 (18000, 1.3140606504024448),
 (22000, 1.1304517058734014),
 (26000, 0.9818072836881715),
 (30000, 0.8901387149131978),
 (34000, 0.8519051070288903),
 (38000, 0.7947895999664009)]